# Reinforcement Learning (RL)
In deze notebook ga ik een eerste simpele kennismaking met Reinforcement Learning doen. Reinforcement Learning, RL in het kort, is een tak van machinelearning/deeplearning waarbij er een **agent** wordt aangemaakt die in een **environment** diverse **actions** onderneemt en hiervoor een reward **krijgt**. Het eerste deel van dit notebook wordt gewijt aan het implementeren van een basale RL agent in een voorafgemaakte environment. Als dit is geslaagd wordt er getracht een eigen agent te programmeren op een custom environment.

## Wat is RL in het kort?
Met reinforcement Learning wordt geprobeerd een agent te trainen de juiste beslissingen te maken. Beslissingen worden gemaakt in een bepaalde omgeving (bijvoorbeeld een lab of een videospel). Om de analogie van het videospel aan te houden: de agent (mario) voert een actie uit (springen) en maakt een observatie (een muntje komt uit een blokje). Deze observaties en acties worden aan elkaar gekoppeld doormiddel van een beloningssysteem. Gedurende veel itteraties zal de agent trainen en het beste resultaat proberen te behalen.

Binnen RL doen we een aantal aannamens of stellen we een paar beperkingen:
* Het probleem is (zeer) complex, anders is RL overkill
* Het betreft een Markov achtige omgeving --> elke actie heeft een reactie ofwel observaties en acties volgen elkaar op
* trainen kan lang duren en is niet altijd stabiel

## Bronnen
[3 hour course](https://www.youtube.com/watch?v=Mut_u40Sqz4)<br>
[baseline docummentatie](https://stable-baselines3.readthedocs.io/en/master/)


In [ ]:
# Python 3.9.7 is nodig voor het gebruik van deze modules
# !pip install stable-baselines3[extra] ## docummentatie voor deze module die RL algoritmes bevat: https://stable-baselines3.readthedocs.io/en/master/
# !pip install pyglet ## extra dependency voor OpenAI gym

In [ ]:
import os # importeren voor het omgaan met paths
import gym # voor het beheren van de omgeving van onze agents
from gym import wrappers # gebruiken om geen visualisaties te doen van de environment
from stable_baselines3 import PPO # een specifiek algorithme voor RL
from stable_baselines3.common.vec_env import DummyVecEnv # wrapper voor environment beheersing
from stable_baselines3.common.evaluation import evaluate_policy # beoordeling van agent policies

## Environments
Voor de environments maken we gebruik van [OpenAI gym](https://gym.openai.com/envs/#classic_control). In deze environments gaat de agent leren door tijdens een aantal episodes "random" acties uit te voeren. De environment is de overkoepelende omgeving waarin de agent acteerd, observaties worden gedaan en waaruit beloning worden gehaald. Een OpenAI gym omgeving kan in verschillende vormen (spaces) voorkomen:
* **Box:** n-dimensionele tensor, die een range van waarden bevat --> Box(0, 1, shape=(3,3))
* **Discrete:** set van discrete waarden --> Discrete(3)
* **Tuple:** Een Tuple van andere spaces --> Tuple((Discrete(3), Box(0, 1, shape=(3,3))))
* **Dict:** Dictionary van spaces --> Dict({('height': Discrete(3), 'speed':Box(0, 1, shape=(3,3)))})
* **MultiBinary:** One hot encoded binary waarde --> MultiBinary(4) --> [0,0,0,0] of [0,1,0,1] etc.
* **MultiDiscrete:** Meerdere discrete waarde --> MultiDiscrete([2,5,2])

In [ ]:
# inladen van environment
environment_name ='CartPole-v0'
env = gym.make(environment_name)
# Test om te zien wat de environment bevat
print(
    f'''
    env.action_space: {env.action_space}
    env.observation_space: {env.observation_space}
    '''
)

## CartPole-v0
Voor de eerste test wordt de CartPole-v0 environment gebruikt. Deze environment bestaat (zoals hierboven te zien) uit:
* Een action space Discrete(2), hierbij is 0 == links en 1 == rechts op een virtuele controller.
* de observation space, het scherm dat de speler/agent ziet is een Box() type de eerste array geeft de lower-bound aan van 4 type observaties, de tweede array de upper-bound. een array eenmaal aangeroepen bestaat uit 4 elementen van het type float32

### De observation space
```
env.observation_space: 
    Box(
        [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38],
        [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38],
        (4,),
        float32)
```

dit kan als volgt geïnterpeteerd worden: <br>
env.observation_space[0] = [-4.8000002e+00] en [4.8000002e+00] --> dit zijn de min en max posities van het wagentje<br>
env.observation_space[1] = [-3.4028235e+38] en [3.4028235e+38] --> de velocity (snelheid) van het wagentje<br>
env.observation_space[2] = [-4.1887903e-01] en [4.1887903e-01] --> hoek van de paal ten opzichten van het wagentje<br>
env.observation_space[3] = [-3.4028235e+38] en [3.4028235e+38] --> velocity van de paal<br>

binnen deze observation space wordt er per env.observation_space[*x*] een waarde teruggeven voor de agent om op te acteren. 

In [ ]:
episodes = 5
 # het aantal keer dat de agent het spel speelt.

for episode in range(0, episodes): # loop over de episodes.
    state = env.reset() # stel de omgeving opnieuw in.
    done = False # zodra het spel is afgelopen verandert dit naar done, en begint dit op false.
    score = 0 # de score die onze agent moet verhogen.

    while not done: # not done == not False == True.
        env.render() #render het spel in een omgeving. normaal gesproken staat dit uit voor snelheid.
        action = env.action_space.sample() # van onze action space kies 1 actie random (links of rechts).
        n_state, reward, done, info = env.step(action) # neem een stap (frame/actie), dit returned 4 waarden. een nieuwe Box(), de beloning, False/True (het spel is afgelopen), en eventueel extra info.
        score += reward # tel de beloning voor deze actie op bij de score van deze episode.
    print(f'Episode: {episode+1} | Score: {score}')
env.close()

## Algorithmes

Er zijn diverse algorithmes algoritmes beschikbaar. Binnen deze notebook wordt gebruik gemaakt van Model-Free RL. dit zijn algorithmes die zich minder/niet bezig houden met het voorspellen van de volgende state maar puur kijken naar een een set regels die de agent maakt op basis van ervaring. Zo kan de agent een regel opstellen dat het wagentje naar links moet als de stok een beetje links kantelt. In onderstaande afbeelding zijn de diverse opties te zien. In dit geval ligt de focus op PPO (Proximal Policy Optimization) dit is een policy algoritme die minder kans heeft om vast te komen zitten in een minima. Normaal gesproken leert policy based algoritmes van de huidige staat en kiest een nieuwe policy. Met PPO gebeurt dit ook maar wordt het verschil tussen policy *p<sub>t</sub>* en *p<sub>t-1</sub>* geminimaliseerd. <br>

![algorithmes](https://spinningup.openai.com/en/latest/_images/rl_algorithms_9_15.svg)

<br>

### Metrics
Voor elk type algoritme zijn diverse metrics beschikbaar. deze kunnen in 4 categoriën worden opgedeeld:
* Evaluation: beschrijving van de lengte en reward voor een episode
* Time: alles wat betreft tijd besteding
* Loss: loss functie om het model te scoren
* Overig: deze worden toeglicht wanneer deze aanbod komen

De metrics worden in apparte mappen opgeslagen als log bestanden en zijn uit te lezen op diverse manieren. In dit notebook wordt gekeken naar de tensorboard visualisaties en .npz files die zullen worden omgezet naar een dataframe.

In [ ]:
# bepaal de locatie voor het opslaan van logs en models
log_path = os.path.join('Training', 'Evaluations')
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
training_path = os.path.join('Training', 'Training')
log_path

In [ ]:
env = gym.make(environment_name) # herhaal het maken van de environment
env = DummyVecEnv([lambda:env]) # wrapper om de environment in een dummy vector te plaatsen
model = PPO('MlpPolicy', env, verbose= 1, tensorboard_log= log_path) # een standaard NeuralNetwork (Multilayer Perceptron), verbose= 1 laat ons de resultaten loggen, tensorboard_log = locatie voor log file

In [ ]:
model.learn(total_timesteps= 20000) # train het model voor x timesteps
model.save(PPO_path) # Sla het model op in de locatie: Training\Saved Models 

In [ ]:
# het model kan geladen worden voor het geval er iets mis is gegaan

try: #probeer eerst het bestaande model te verwijderen uit de variable list en laad dan een opgeslagen model
    del model #verwijder het bestaande model
    model = PPO.load(PPO_path, env=env) # laad het model
except NameError: # als het model niet bestaat bij de "del model" functie catch the exception en laad het model
    model = PPO.load(PPO_path, env=env)

print(f'Model is ingeladen!')

## Evalueren van model en tensorboard
In de onderstaande cellen gaan we het model evalueren en de de geloggde gegevens in een tensorboard plaatsen. 
Voor nu zijn de belangrijkste metrics om in de gaten te houden de *average Reward* en *average episode length*.
Aangezien deze niet aanwezig zijn in de tensorboard worden deze appart gevisualiseerd.


In [ ]:
# evalueer het model op basis van een aantal parameters (average punten, standard deviation) --> max punten = 200.
evaluate_policy(model, env, n_eval_episodes= 10, render=True) 
env.close()

In [ ]:
# draai nu de zelfde environment met een getrainde agent
def run_agent(model, episodes= 5, render = False):
    '''
    runs an agent (model) through the environment for n episodes, if render is set to False no window is rendered.
    at the end of the runs a tensorlog file is written to a predetermined directory
    '''
    for episode in range(0, episodes): 
        obs = env.reset() 
        done = False 
        score = 0 

        while not done:
            if render: 
                env.render(mode='human') 
            action, _ = model.predict(obs) # inplaats van een random sample van de mogelijke acties maakt het model een keuze op basis van de huidige observatie, de eerste waarde is de actie, de tweede waarde '_' is de state
            obs, reward, done, info = env.step(action)
            score += reward
        print(f'Episode: {episode+1} | Score: {score}')
    env.close()

run_agent(model, render= True)

In [ ]:
#Tensorlog ophalen van een locatie
train_log_path = os.path.join(log_path, 'PPO_3')

#klik de play button hieronder
%tensorboard --logdir= train_log_path # klik de play button hierboven voor de Tensorboard evaluations grafieken

## Evalueren op basis van score

In de onderstaande cellen wordt de agent gescoord op zijn vaardigheden. Gezien de omgeving redelijk simpel is vindt de agent over het algemeen met gemak een optimale oplossing na verloop van het trainings proces.

De eerst volgende 2 cellen beoordelen een volledig getrained model. Het is duidelijk dat hier weinig interessants uitkomt. Het model scoort namelijk in 4 van de 5 episodes maximaal.

In [ ]:
# Laad het model
model = PPO.load(PPO_path, env=env)

episodes = 5 
lst_score = []
lst_episode = []

#Draai het model
for episode in range(0, episodes): 
    obs = env.reset() 
    done = False 
    score = 0 

    while not done: 
        env.render() 
        action, _ = model.predict(obs) # inplaats van een random sample van de mogelijke acties maakt het model een keuze op basis van de huidige observatie, de eerste waarde is de actie, de tweede waarde '_' is de state
        obs, reward, done, info = env.step(action)
        score += reward
    lst_score.append(score[0])
    lst_episode.append(episode + 1)
env.close()

In [ ]:
import pandas as pd # dataframe en manipulatie
import seaborn as sns # visualisatie
from matplotlib import pyplot as plt # extra functionaliteit voor visualisatie

d = {"episode": lst_episode, "score": lst_score}
df = pd.DataFrame(d)
df

sns.lineplot(x= df["episode"], y=df["score"])

Hieronder worden 2 nieuwe agents getrained. Wordt de evaluatie van de agent tijdens het trainen gelogged naar een .npz file. In deze log komen alleen de scores en de duratie van de episodes terug.

Door twee verschillende modellen te trainen kunnen we het effect van PPO policy duidelijk terug zien. Het is namelijk zo dat de PPO minimale aanpassingen doet in de voorgaande strategie (timestep). Deze afwijkingen zorgen doorgaans voor een langzaam stijgende lijn en voorkomt in de meeste gevallen een foute strategie door een te grote policy wijziging. In de 3e cell hieronder is dit ook te zien. PPO zorgt voor een doorgaans stijgende lijn en afwijkingen in de policy worden minimaal gemaakt hierdoor zie je dat een agent niet snel in een local optimum komt of naar een minima toe beweegt. Hier moet wel gemeld worden dat dit een zeer simpele omgeving betreft waardoor het niet vreemd is dat agents in deze omgeving een globaal optimum bereiken.

![model1](visualisatie\model1.png)

In [ ]:
#nieuw model voor loggen trainings informatie
# env = wrappers.Monitor(env, video_callable=False ,force=True)
model2 = PPO('MlpPolicy', env)
model2.learn(total_timesteps= 20000, eval_freq  =100, eval_log_path = training_path, eval_env= env, n_eval_episodes= 5)


In [ ]:
import numpy as np # gebruiken voor wiskundige operaties en npz files te lezen
npz = np.load('Training\Training\evaluations.npz')

def npz_to_dataframe(npz):
    '''
    Takes an npz file object loaded using numpy.load(path to .npz file) and returns a formatted dataframe
    '''
    # maak een data frame van de npz file op basis van de kolommen
    df = pd.DataFrame.from_dict({col: npz[col] for col in npz.files}, orient='index')
    # display(df_training_score) #-->uncomment om de transformatie te zien

    # transpose de dataframe gezien deze verkeerd om staat
    df =df.T
    # display(df_training_score) #-->uncomment om de transformatie te zien

    # bereken de gemiddeldes voor iedere cel
    df = df.applymap(np.mean)
    # display(df_training_score) #-->uncomment om de transformatie te zien
    return df

df_training_score = npz_to_dataframe(npz)
df_training_score

In [ ]:
# Maak een lineplot om de prestaties van het model na diverse timesteps te zien
fig = sns.lineplot(x= df_training_score["timesteps"], y=df_training_score["results"])
display(fig)
fig = fig.get_figure()
fig.savefig('visualisatie\model1')


In [ ]:
# # vergelijk de training met een tweede model
model3 = PPO('MlpPolicy', env)
model3.learn(total_timesteps= 20000, eval_freq  =100, eval_log_path = training_path, eval_env= env, n_eval_episodes= 5)

npz = np.load('Training\Training\evaluations.npz')

# Roep de eerder gemaakte functie aan
df_training_score2 = npz_to_dataframe(npz)

#voeg aan beide data frames een kolom toe met de agent naam
df_training_score["model"] = "model 1"
df_training_score2["model"] = "model 2"

Hieronder worden de twee agents tijdens het trainen met elkaar vergeleken. Het is goed te zien dat de twee agents andere start punten hebben. Door kleine aanpassingen in de strategie kan de score in het begin sterk stijgen/dalen om vervolgens na een redelijk stijgende lijn in een bepaald optimum te settlen.

In [ ]:
df_compare = df_training_score.append(df_training_score2)
fig = sns.lineplot(x= df_compare["timesteps"], y=df_compare["results"], hue= df_compare['model'])
display(fig)

fig = fig.get_figure()
fig.savefig('visualisatie\model1vsmodel2')